In [1]:
!pip install tsundoku

In [2]:
!pip install sklearn torch tqdm nltk lazyme requests gensim
!python -m nltk.downloader movie_reviews

  Stored in directory: /Users/davidleonardi/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
  Stored in directory: /Users/davidleonardi/Library/Caches/pip/wheels/85/7a/38/7af821d7ef46e04c7c74cc033e26fd7487572c4609331c437c
Successfully built sklearn lazyme
/anaconda3/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/davidleonardi/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [3]:
from IPython.display import display, Markdown, Latex
from tsundoku.word2vec_hints import *

# Overview


- <a href="#section-3-0">**3.0. Data Preparation**</a>
  - <a href="#section-3-0-1">3.0.1. *Vocabulary*</a>
    - <a href="#section-3-0-1-a"> Pet Peeve: using `gensim`</a>
  - <a href="#section-3-0-2">3.0.2. *Dataset*</a>  (<a href="#section-3-0-2-hints">Hints</a>)
    - <a href="#section-3-0-2-return-dict">Return `dict` in `__getitem__()`</a>
    - <a href="#section-3-0-2-labeleddata">Try `LabeledDataset`</a>
<br><br>
- <a href="#section-3-1">**3.1. Word2Vec from Scratch**</a>
  - <a href="#section-3-1-1">3.1.1. *CBOW*</a>
  - <a href="#section-3-1-2">3.1.2. *Skipgram*</a>
  - <a href="#section-3-1-3">3.1.3. *Word2Vec Dataset*</a> (<a href="#section-3-1-3-hint">Hints</a>)
  - <a href="#section-3-1-4-hint">3.1.4. *Train a CBOW model*</a>
    - <a href="#section-3-1-4-fill-cbow">Fill in the CBOW model</a>
    - <a href="#section-3-1-4-train-cbow">Train the model (*for real*)</a>
    - <a href="#section-3-1-4-evaluate-cbow">Evaluate the model</a>
    - <a href="#section-3-1-4-load-model">Load model at specific epoch</a>
  - <a href="#section-3-1-5">3.1.5. *Train a Skipgram model*</a>
    - <a href="#section-3-1-5-forward">Take a closer look at `forward()`</a>
    - <a href="#section-3-1-5-train">Train the model (*for real*)</a>
    - <a href="section-3-1-5-evaluate">Evaluate the model</a>
  - <a href="#section-3-1-6">3.1.6. *Loading Pre-trained Embeddings*</a>
    - <a href="#section-3-1-6-vocab">Override the Embedding vocabulary</a>
    - <a href="#section-3-1-6-pretrained">Override the Embedding weights</a>
    - <a href="#section-3-1-6-eval-skipgram">Evaluate on the Skipgram task</a>
    - <a href="#section-3-1-6-eval-cbow">Evaluate on the CBOW task</a>
    - <a href="#section-3-1-6-unfreeze-finetune">Unfreeeze and finetune</a>
    - <a href="#section-3-1-6-reval-cbow">Re-evaluate on the CBOW task</a>
<br><br>


<a id="section-3-0"></a>
# 3.0. Data Preparation

Before we train our own embeddings, lets first understand how to read text data into pytorch.
The native pytorch way to load datasets is to use the `torch.utils.data.Dataset` object.

There are already several other libraries that help with loading text datasets, e.g. 

 - FastAI https://docs.fast.ai/text.data.html
 - AllenNLP https://allenai.github.io/allennlp-docs/api/allennlp.data.dataset.html
 - Torch Text https://github.com/pytorch/text#data
 - Texar https://texar.readthedocs.io/en/latest/code/data.html#id4 
 - SpaCy https://github.com/explosion/thinc
 

But to truly understand and use it for the custom datasets you'll see at work, lets learn it the native way.

<a id="section-3-0-1"></a>
## 3.0.1  Vocabulary

Given a text, the first thing to do is to build a vocabulary (i.e. a dictionary of unique words) and assign an index to each unique word.

In [4]:
import random
from itertools import chain

from tqdm import tqdm
from gensim.corpora import Dictionary

import torch
from torch import nn, optim, tensor, autograd
from torch.nn import functional as F
from torch.utils.data import Dataset
import numpy as np

from functools import partial
from torch.utils.data import Dataset, DataLoader
from torch import functional as F


In [5]:
try: # Use the default NLTK tokenizer.
    from nltk import word_tokenize, sent_tokenize 
    # Testing whether it works. 
    # Sometimes it doesn't work on some machines because of setup issues.
    word_tokenize(sent_tokenize("This is a foobar sentence. Yes it is.")[0])
except: # Use a naive sentence tokenizer and toktok.
    import re
    from nltk.tokenize import ToktokTokenizer
    # See https://stackoverflow.com/a/25736515/610569
    sent_tokenize = lambda x: re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', x)
    # Use the toktok tokenizer that requires no dependencies.
    toktok = ToktokTokenizer()
    word_tokenize = word_tokenize = toktok.tokenize


In [6]:

text = """Language users never choose words randomly, and language is essentially
non-random. Statistical hypothesis testing uses a null hypothesis, which
posits randomness. Hence, when we look at linguistic phenomena in corpora, 
the null hypothesis will never be true. Moreover, where there is enough
data, we shall (almost) always be able to establish that it is not true. In
corpus studies, we frequently do have enough data, so the fact that a relation 
between two phenomena is demonstrably non-random, does not support the inference 
that it is not arbitrary. We present experimental evidence
of how arbitrary associations between word frequencies and corpora are
systematically non-random. We review literature in which hypothesis testing 
has been used, and show how it has often led to unhelpful or misleading results.""".lower()

tokenized_text = [word_tokenize(sent) for sent in sent_tokenize(text)]

uniq_tokens = set(chain(*tokenized_text))

vocab = {}   # Assign indices to every word.
idx2tok = {} # Also keep an dict of index to words.
for i, token in enumerate(uniq_tokens):
    vocab[token] = i
    idx2tok[i] = token

In [7]:
vocab

{'(': 44,
 ')': 52,
 ',': 34,
 '.': 47,
 'a': 18,
 'able': 31,
 'almost': 27,
 'always': 19,
 'and': 8,
 'arbitrary': 3,
 'are': 56,
 'associations': 38,
 'at': 12,
 'be': 2,
 'been': 76,
 'between': 62,
 'choose': 1,
 'corpora': 10,
 'corpus': 75,
 'data': 73,
 'demonstrably': 60,
 'do': 11,
 'does': 37,
 'enough': 69,
 'essentially': 79,
 'establish': 82,
 'evidence': 81,
 'experimental': 0,
 'fact': 48,
 'frequencies': 40,
 'frequently': 9,
 'has': 70,
 'have': 74,
 'hence': 28,
 'how': 25,
 'hypothesis': 33,
 'in': 50,
 'inference': 54,
 'is': 85,
 'it': 46,
 'language': 68,
 'led': 45,
 'linguistic': 21,
 'literature': 58,
 'look': 22,
 'misleading': 5,
 'moreover': 80,
 'never': 35,
 'non-random': 4,
 'not': 32,
 'null': 72,
 'of': 71,
 'often': 53,
 'or': 14,
 'phenomena': 30,
 'posits': 65,
 'present': 13,
 'randomly': 55,
 'randomness': 86,
 'relation': 77,
 'results': 66,
 'review': 41,
 'shall': 26,
 'show': 7,
 'so': 36,
 'statistical': 61,
 'studies': 59,
 'support': 49,
 

In [9]:
# Retrieve the index of the word 'corpora'
vocab['corpora']

10

In [10]:
# The indexed representation of the first sentence.

sent0 = tokenized_text[0]

[vocab[token] for token in sent0] 

[68, 6, 35, 1, 24, 55, 34, 8, 68, 85, 79, 4, 47]

<a id="section-3-0-1-a"></a>

### Pet Peeve (Gensim)

I (Liling) don't really like to write my own vectorizer the `gensim` has functions that are optimized for such operations. In fact, I've written a [whole preprocessing pipeline library for me to use for language modelling and machine translation purposes](https://github.com/alvations/komorebi/blob/master/komorebi/text.py) =)

Using `gensim`, I would have written the above as such:

In [11]:
from gensim.corpora.dictionary import Dictionary
vocab = Dictionary(tokenized_text)

In [12]:
# Note the key-value order is different of gensim from the native Python's
dict(vocab.items())

{0: ',',
 1: '.',
 2: 'and',
 3: 'choose',
 4: 'essentially',
 5: 'is',
 6: 'language',
 7: 'never',
 8: 'non-random',
 9: 'randomly',
 10: 'users',
 11: 'words',
 12: 'a',
 13: 'hypothesis',
 14: 'null',
 15: 'posits',
 16: 'randomness',
 17: 'statistical',
 18: 'testing',
 19: 'uses',
 20: 'which',
 21: 'at',
 22: 'be',
 23: 'corpora',
 24: 'hence',
 25: 'in',
 26: 'linguistic',
 27: 'look',
 28: 'phenomena',
 29: 'the',
 30: 'true',
 31: 'we',
 32: 'when',
 33: 'will',
 34: '(',
 35: ')',
 36: 'able',
 37: 'almost',
 38: 'always',
 39: 'data',
 40: 'enough',
 41: 'establish',
 42: 'it',
 43: 'moreover',
 44: 'not',
 45: 'shall',
 46: 'that',
 47: 'there',
 48: 'to',
 49: 'where',
 50: 'arbitrary',
 51: 'between',
 52: 'corpus',
 53: 'demonstrably',
 54: 'do',
 55: 'does',
 56: 'fact',
 57: 'frequently',
 58: 'have',
 59: 'inference',
 60: 'relation',
 61: 'so',
 62: 'studies',
 63: 'support',
 64: 'two',
 65: 'are',
 66: 'associations',
 67: 'evidence',
 68: 'experimental',
 69: 'fr

In [13]:
vocab.token2id['corpora']

23

In [14]:
vocab.doc2idx(sent0)

[6, 10, 7, 3, 11, 9, 0, 2, 6, 5, 4, 8, 1]

The "indexed form" of the tokens in the sentence forms the ***vectorized*** input to the `nn.Embedding` layer in PyTorch.

<a id="section-3-0-2"></a>

# 3.0.2 Dataset

Lets try creating a `torch.utils.data.Dataset` object.

In [15]:
from torch.utils.data import Dataset, DataLoader

class Text(Dataset):
    def __init__(self, tokenized_texts):
        """
        :param tokenized_texts: Tokenized text.
        :type tokenized_texts: list(list(str))
        """
        self.sents = tokenized_texts
        self.vocab = Dictionary(tokenized_texts)

    def __getitem__(self, index):
        """
        The primary entry point for PyTorch datasets.
        This is were you access the specific data row you want.
        
        :param index: Index to the data point.
        :type index: int
        """
        # Hint: You want to return a vectorized sentence here.
        return {'x': self.vectorize(self.sents[index])}

    def vectorize(self, tokens):
        """
        :param tokens: Tokens that should be vectorized. 
        :type tokens: list(str)
        """
        # See https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2idx 
        return self.vocab.doc2idx(tokens)
    
    def unvectorize(self, indices):
        """
        :param indices: Converts the indices back to tokens.
        :type tokens: list(int)
        """
        return [self.vocab[i] for i in indices]

<a id="section-3-0-2-hints"></a>
## Hints to the above cell

In [16]:
# Option 1: To see the hint and partial code for the cell above, uncomment the following line.
##hint_dataset_vectorize()
##code_text_dataset_vectorize()

# Option 2: "I give up just, run the code for me" 
# Uncomment the next two lines, if you really gave up... 
#full_code_text_dataset_vectorize()
##from tsundoku.word2vec import Text


In [17]:
tokenized_text[5]

['we',
 'present',
 'experimental',
 'evidence',
 'of',
 'how',
 'arbitrary',
 'associations',
 'between',
 'word',
 'frequencies',
 'and',
 'corpora',
 'are',
 'systematically',
 'non-random',
 '.']

In [18]:
text_dataset = Text(tokenized_text)

In [19]:
text_dataset[5] # First sentence.

{'x': [31, 72, 68, 67, 71, 70, 50, 66, 51, 74, 69, 2, 23, 65, 73, 8, 1]}

<a id="section-3-0-2-return-dict"></a>

### Return `dict` in `__getitem__()`

This is nice if we're just representing sentences/documents by their indices but when we're doing machine learning, we usually have `X` and `Y`. 

If we have labels for the each sentence, we can also put it into to `__getitem__()` by having it return a dictionary.

In [20]:
from torch.utils.data import Dataset, DataLoader

class LabeledText(Dataset):
    def __init__(self, tokenized_texts, labels):
        """
        :param tokenized_texts: Tokenized text.
        :type tokenized_texts: list(list(str))
        """
        self.sents = tokenized_texts
        self.labels = labels # Sentence level labels.
        self.vocab = Dictionary(self.sents)

    def __getitem__(self, index):
        """
        The primary entry point for PyTorch datasets.
        This is were you access the specific data row you want.
        
        :param index: Index to the data point.
        :type index: int
        """
        return {'X': self.vectorize(self.sents[index]), 'Y': self.labels[index]}

    def vectorize(self, tokens):
        """
        :param tokens: Tokens that should be vectorized. 
        :type tokens: list(str)
        """
        # See https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2idx 
        return self.vocab.doc2idx(tokens)
    
    def unvectorize(self, indices):
        """
        :param indices: Converts the indices back to tokens.
        :type tokens: list(int)
        """
        return [self.vocab[i] for i in indices]

<a id="section-3-0-2-labeleddata"></a>

### Lets try the `LabeledDataset` on a movie review corpus

In [21]:
from nltk.corpus import movie_reviews

In [22]:
documents = []
labels = []

for fileid in tqdm(movie_reviews.fileids()):
    label = fileid.split('/')[0]
    doc = word_tokenize(movie_reviews.open(fileid).read())
    documents.append(doc)
    labels.append(label)

100%|██████████| 2000/2000 [00:11<00:00, 175.02it/s]


In [23]:
documents[0]

['plot',
 ':',
 'two',
 'teen',
 'couples',
 'go',
 'to',
 'a',
 'church',
 'party',
 ',',
 'drink',
 'and',
 'then',
 'drive',
 '.',
 'they',
 'get',
 'into',
 'an',
 'accident',
 '.',
 'one',
 'of',
 'the',
 'guys',
 'dies',
 ',',
 'but',
 'his',
 'girlfriend',
 'continues',
 'to',
 'see',
 'him',
 'in',
 'her',
 'life',
 ',',
 'and',
 'has',
 'nightmares',
 '.',
 'what',
 "'s",
 'the',
 'deal',
 '?',
 'watch',
 'the',
 'movie',
 'and',
 '``',
 'sorta',
 '``',
 'find',
 'out',
 '.',
 '.',
 '.',
 'critique',
 ':',
 'a',
 'mind-fuck',
 'movie',
 'for',
 'the',
 'teen',
 'generation',
 'that',
 'touches',
 'on',
 'a',
 'very',
 'cool',
 'idea',
 ',',
 'but',
 'presents',
 'it',
 'in',
 'a',
 'very',
 'bad',
 'package',
 '.',
 'which',
 'is',
 'what',
 'makes',
 'this',
 'review',
 'an',
 'even',
 'harder',
 'one',
 'to',
 'write',
 ',',
 'since',
 'i',
 'generally',
 'applaud',
 'films',
 'which',
 'attempt',
 'to',
 'break',
 'the',
 'mold',
 ',',
 'mess',
 'with',
 'your',
 'head',
 '

In [24]:
labeled_dataset = LabeledText(documents, labels)

In [25]:
labeled_dataset[0]  # First review in the data.

{'X': [243,
  17,
  314,
  294,
  77,
  140,
  307,
  20,
  68,
  237,
  6,
  97,
  34,
  299,
  98,
  8,
  302,
  135,
  167,
  33,
  22,
  8,
  226,
  220,
  297,
  145,
  87,
  6,
  60,
  158,
  136,
  74,
  307,
  262,
  157,
  165,
  153,
  179,
  6,
  34,
  149,
  214,
  8,
  333,
  2,
  297,
  82,
  18,
  326,
  297,
  204,
  34,
  19,
  280,
  19,
  124,
  230,
  8,
  8,
  8,
  79,
  17,
  20,
  199,
  204,
  129,
  297,
  294,
  133,
  296,
  311,
  225,
  20,
  322,
  75,
  164,
  6,
  60,
  245,
  169,
  165,
  20,
  322,
  46,
  234,
  8,
  337,
  168,
  333,
  188,
  304,
  253,
  33,
  108,
  148,
  226,
  307,
  345,
  6,
  272,
  163,
  132,
  37,
  122,
  337,
  42,
  307,
  59,
  297,
  201,
  6,
  196,
  341,
  348,
  152,
  34,
  290,
  4,
  185,
  156,
  1,
  195,
  5,
  6,
  60,
  300,
  38,
  142,
  34,
  46,
  328,
  220,
  189,
  28,
  315,
  220,
  122,
  6,
  34,
  301,
  128,
  173,
  86,
  208,
  276,
  304,
  226,
  76,
  8,
  302,
  263,
  307,
  150,
  2

In [26]:
labeled_dataset[0]['X']  # Label of the first review in the data. 

[243,
 17,
 314,
 294,
 77,
 140,
 307,
 20,
 68,
 237,
 6,
 97,
 34,
 299,
 98,
 8,
 302,
 135,
 167,
 33,
 22,
 8,
 226,
 220,
 297,
 145,
 87,
 6,
 60,
 158,
 136,
 74,
 307,
 262,
 157,
 165,
 153,
 179,
 6,
 34,
 149,
 214,
 8,
 333,
 2,
 297,
 82,
 18,
 326,
 297,
 204,
 34,
 19,
 280,
 19,
 124,
 230,
 8,
 8,
 8,
 79,
 17,
 20,
 199,
 204,
 129,
 297,
 294,
 133,
 296,
 311,
 225,
 20,
 322,
 75,
 164,
 6,
 60,
 245,
 169,
 165,
 20,
 322,
 46,
 234,
 8,
 337,
 168,
 333,
 188,
 304,
 253,
 33,
 108,
 148,
 226,
 307,
 345,
 6,
 272,
 163,
 132,
 37,
 122,
 337,
 42,
 307,
 59,
 297,
 201,
 6,
 196,
 341,
 348,
 152,
 34,
 290,
 4,
 185,
 156,
 1,
 195,
 5,
 6,
 60,
 300,
 38,
 142,
 34,
 46,
 328,
 220,
 189,
 28,
 315,
 220,
 122,
 6,
 34,
 301,
 128,
 173,
 86,
 208,
 276,
 304,
 226,
 76,
 8,
 302,
 263,
 307,
 150,
 293,
 304,
 246,
 209,
 72,
 6,
 60,
 113,
 169,
 295,
 8,
 277,
 333,
 38,
 297,
 248,
 341,
 297,
 204,
 18,
 331,
 6,
 170,
 186,
 247,
 168,
 296,
 169,
 2,

<a id="section-3-1"></a>

# 3.1 Word2Vec Training

Word2Vec has two training variants:

 - **Continuous Bag of Words (CBOW)**: Predict center word from (bag of) context words.
 - **Skip-grams**: Predict context words given center word.
  
Visually, they look like this:

<img src="https://ibin.co/4UIznsOEyH7t.png" width="500" align="left">


<a id="section-3-1-1"></a>

## 3.1.1. CBOW

CBOW windows through the sentence and picks out the center word as the `Y` and the surrounding context words as the inputs `X`. 


In [27]:
from lazyme import per_window, per_chunk

xx =[1,2,3,4]
list(per_window(xx, n=2))
list(per_chunk(xx, n=3))

[(1, 2, 3), (4, None, None)]

In [28]:
def per_window(sequence, n=1):
    """
    From http://stackoverflow.com/q/42220614/610569
        >>> list(per_window([1,2,3,4], n=2))
        [(1, 2), (2, 3), (3, 4)]
        >>> list(per_window([1,2,3,4], n=3))
        [(1, 2, 3), (2, 3, 4)]
    """
    start, stop = 0, n
    seq = list(sequence)
    while stop <= len(seq):
        yield seq[start:stop]
        start += 1
        stop += 1

def cbow_iterator(tokens, window_size):
    n = window_size * 2 + 1
    for window in per_window(tokens, n):
        target = window.pop(window_size)
        yield window, target   # X = window ; Y = target. 


In [29]:
sent0 = ['language', 'users', 'never', 'choose', 'words', 'randomly', ',', 
         'and', 'language', 'is', 'essentially', 'non-random', '.']

In [30]:
list(cbow_iterator(sent0, 2)) 

[(['language', 'users', 'choose', 'words'], 'never'),
 (['users', 'never', 'words', 'randomly'], 'choose'),
 (['never', 'choose', 'randomly', ','], 'words'),
 (['choose', 'words', ',', 'and'], 'randomly'),
 (['words', 'randomly', 'and', 'language'], ','),
 (['randomly', ',', 'language', 'is'], 'and'),
 ([',', 'and', 'is', 'essentially'], 'language'),
 (['and', 'language', 'essentially', 'non-random'], 'is'),
 (['language', 'is', 'non-random', '.'], 'essentially')]

In [31]:
list(cbow_iterator(sent0, 3)) 

[(['language', 'users', 'never', 'words', 'randomly', ','], 'choose'),
 (['users', 'never', 'choose', 'randomly', ',', 'and'], 'words'),
 (['never', 'choose', 'words', ',', 'and', 'language'], 'randomly'),
 (['choose', 'words', 'randomly', 'and', 'language', 'is'], ','),
 (['words', 'randomly', ',', 'language', 'is', 'essentially'], 'and'),
 (['randomly', ',', 'and', 'is', 'essentially', 'non-random'], 'language'),
 ([',', 'and', 'language', 'essentially', 'non-random', '.'], 'is')]

<a id="section-3-1-2"></a>

## 3.1.2. Skipgram

Skipgram training windows through the sentence and pictures out the center word as the input `X` and the context words as the outputs `Y`, additionally, it will randommly sample words not in the window as **negative samples**.

In [32]:
def skipgram_iterator(tokens, window_size):
    n = window_size * 2 + 1 
    for i, window in enumerate(per_window(tokens, n)):
        target = window.pop(window_size)
        # Generate positive samples.
        for context_word in window:
            yield target, context_word, 1
        # Generate negative samples.
        for _ in range(n-1):
            leftovers = tokens[:i] + tokens[i+n:]
            yield target, random.choice(leftovers), 0

In [33]:
list(skipgram_iterator(sent0, 2))

[('never', 'language', 1),
 ('never', 'users', 1),
 ('never', 'choose', 1),
 ('never', 'words', 1),
 ('never', 'essentially', 0),
 ('never', '.', 0),
 ('never', 'essentially', 0),
 ('never', 'essentially', 0),
 ('choose', 'users', 1),
 ('choose', 'never', 1),
 ('choose', 'words', 1),
 ('choose', 'randomly', 1),
 ('choose', 'and', 0),
 ('choose', 'essentially', 0),
 ('choose', 'non-random', 0),
 ('choose', 'is', 0),
 ('words', 'never', 1),
 ('words', 'choose', 1),
 ('words', 'randomly', 1),
 ('words', ',', 1),
 ('words', 'language', 0),
 ('words', 'essentially', 0),
 ('words', 'is', 0),
 ('words', 'language', 0),
 ('randomly', 'choose', 1),
 ('randomly', 'words', 1),
 ('randomly', ',', 1),
 ('randomly', 'and', 1),
 ('randomly', 'users', 0),
 ('randomly', 'essentially', 0),
 ('randomly', 'language', 0),
 ('randomly', 'essentially', 0),
 (',', 'words', 1),
 (',', 'randomly', 1),
 (',', 'and', 1),
 (',', 'language', 1),
 (',', 'users', 0),
 (',', 'essentially', 0),
 (',', 'users', 0),
 (',

## Cut-away: What is `partial`?

The [`functools.partial`](https://docs.python.org/3.7/library/functools.html#functools.partial) function in Python is a mechanism to overload a function with preset arguments. 

For example:

In [34]:
from nltk import ngrams

# Generates bigrams
list(ngrams('this is a sentence'.split(), n=2))

[('this', 'is'), ('is', 'a'), ('a', 'sentence')]

In [35]:
from functools import partial

# You can create a new function that "preset" the `n` argument, e.g.
bigrams = partial(ngrams, n=2)
trigrams = partial(ngrams, n=3)

In [36]:
list(trigrams('this is a sentence'.split()))

[('this', 'is', 'a'), ('is', 'a', 'sentence')]

In [37]:
list(bigrams('this is a sentence'.split()))

[('this', 'is'), ('is', 'a'), ('a', 'sentence')]

<a id="section-3-1-3"></a>

## 3.1.3 Word2Vec Dataset

Now that we know what are the inputs `X` and outputs `Y` of the Word2Vec task. 

Lets put everything together and modify the `Dataset` so that `__getitem__` retrieves CBOW or Skipgram formats.

In [38]:
class Word2VecText(Dataset):
    def __init__(self, tokenized_texts, window_size, variant):
        """
        :param tokenized_texts: Tokenized text.
        :type tokenized_texts: list(list(str))
        """
        self.sents = tokenized_texts
        self._len = len(self.sents)
        self.vocab = Dictionary(self.sents)
        self.window_size = window_size
        self.variant = variant
        if variant.lower() == 'cbow':
            self._iterator = partial(self.cbow_iterator, window_size=self.window_size)
        elif variant.lower() == 'skipgram':
            self._iterator = partial(self.skipgram_iterator, window_size=self.window_size)

    def __getitem__(self, index):
        """
        The primary entry point for PyTorch datasets.
        This is were you access the specific data row you want.
        
        :param index: Index to the data point.
        :type index: int
        """
        vectorized_sent = self.vectorize(self.sents[index])
        
        return list(self._iterator(vectorized_sent))

    def __len__(self):
        return self._len
    
    def vectorize(self, tokens):
        """
        :param tokens: Tokens that should be vectorized. 
        :type tokens: list(str)
        """
        # See https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2idx 
        return self.vocab.doc2idx(tokens)
    
    def unvectorize(self, indices):
        """
        :param indices: Converts the indices back to tokens.
        :type tokens: list(int)
        """
        return [self.vocab[i] for i in indices]

    def cbow_iterator(self,tokens, window_size):
        n = window_size * 2 + 1
        for window in per_window(tokens, n):
            target = window.pop(window_size)
            yield window, target   # X = window ; Y = target. 

    def skipgram_iterator(self, tokens, window_size):
        n = window_size * 2 + 1 
        for i, window in enumerate(per_window(tokens, n)):
            target = window.pop(window_size)
            # Generate positive samples.
            for context_word in window:
                yield target, context_word, 1
            # Generate negative samples.
            for _ in range(n-1):
                leftovers = tokens[:i] + tokens[i+n:]
                yield target, random.choice(leftovers), 0
                

<a id="section-3-1-3-hint"></a>
## Hints for the cell above.

In [39]:
# Option 1: To see the hint and partial code for the cell above, uncomment the following line.
##hint_word2vec_dataset()

# Option 2: "I give up just, run the code for me" 
# Uncomment the next two lines, if you really gave up... 
##full_code_word2vec_dataset()
##from tsundoku.word2vec import Word2VecText


<a id="section-3-1-4-hint"></a>

## 3.1.4. Train a CBOW model

### Lets Get Some Data

Lets take Kilgarriff (2005) , "Language is never ever, ever random". 

In [40]:
import os
import requests
import io #codecs


# Text version of https://kilgarriff.co.uk/Publications/2005-K-lineer.pdf
if os.path.isfile('language-never-random.txt'):
    with io.open('language-never-random.txt', encoding='utf8') as fin:
        text = fin.read()
else:
    url = "https://gist.githubusercontent.com/alvations/53b01e4076573fea47c6057120bb017a/raw/b01ff96a5f76848450e648f35da6497ca9454e4a/language-never-random.txt"
    text = requests.get(url).content.decode('utf8')
    with io.open('language-never-random.txt', 'w', encoding='utf8') as fout:
        fout.write(text)

tokenized_text = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(text)]
window_size = 2
w2v_dataset = Word2VecText(tokenized_text, window_size=window_size, variant='cbow')

In [41]:
print(text[:1000])

                       Language is never, ever, ever, random

                                                               ADAM KILGARRIFF




Abstract
Language users never choose words randomly, and language is essentially
non-random. Statistical hypothesis testing uses a null hypothesis, which
posits randomness. Hence, when we look at linguistic phenomena in cor-
pora, the null hypothesis will never be true. Moreover, where there is enough
data, we shall (almost) always be able to establish that it is not true. In
corpus studies, we frequently do have enough data, so the fact that a rela-
tion between two phenomena is demonstrably non-random, does not sup-
port the inference that it is not arbitrary. We present experimental evidence
of how arbitrary associations between word frequencies and corpora are
systematically non-random. We review literature in which hypothesis test-
ing has been used, and show how it has often led to unhelpful or mislead-
ing results.
Keywords: 쎲쎲쎲

1. Int

In [42]:
# Sanity check, lets take a look at the data.
print(tokenized_text[0])

['language', 'is', 'never', ',', 'ever', ',', 'ever', ',', 'random', 'adam', 'kilgarriff', 'abstract', 'language', 'users', 'never', 'choose', 'words', 'randomly', ',', 'and', 'language', 'is', 'essentially', 'non-random', '.']


In [48]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [49]:
from lazyme import color_str

def visualize_predictions(x, y, prediction, vocab, window_size, unk='<unk>'):
    left = ' '.join([vocab.get(int(_x), '<unk>') for _x in x[:window_size]])
    right = ' '.join([vocab.get(int(_x), '<unk>') for _x in x[window_size:]])
    target = vocab.get(int(y), '<unk>')

    if not prediction:
        predicted_word = '______'
    else:
        predicted_word = vocab.get(int(prediction), '<unk>') 
    print(color_str(target, 'green'), '\t' if len(target) > 6 else '\t\t', 
          left, color_str(predicted_word, 'green' if target == predicted_word else 'red'), right)
    

sent_idx = 10
window_size = 2
w2v_dataset = Word2VecText(tokenized_text, window_size=window_size, variant='cbow')
print(' '.join(w2v_dataset.sents[sent_idx]))
for w2v_io in w2v_dataset[sent_idx]:
    context, target = w2v_io
    context, target = tensor(context).to(device), tensor(target).to(device)
    visualize_predictions(context, target, None, w2v_dataset.vocab, window_size)

the bulk of linguistic questions concern the dis- tinction between a and m. a linguistic account of a phenomenon gen- erally gives us reason to view the relation between , for example , a verb ’ s syntax and its semantics , as motivated rather than arbitrary .
of 		 the bulk ______ linguistic questions
linguistic 	 bulk of ______ questions concern
questions 	 of linguistic ______ concern the
concern 	 linguistic questions ______ the dis-
the 		 questions concern ______ dis- tinction
dis- 		 concern the ______ tinction between
tinction 	 the dis- ______ between a
between 	 dis- tinction ______ a and
a 		 tinction between ______ and m.
and 		 between a ______ m. a
m. 		 a and ______ a linguistic
a 		 and m. ______ linguistic account
linguistic 	 m. a ______ account of
account 	 a linguistic ______ of a
of 		 linguistic account ______ a phenomenon
a 		 account of ______ phenomenon gen-
phenomenon 	 of a ______ gen- erally
gen- 		 a phenomenon ______ erally gives
erally 		 phenomenon gen- 

<a id="section-3-1-4-cbow-model"></a>

## Fill-in the code for the CBOW Model

<img src="https://lilianweng.github.io/lil-log/assets/images/word2vec-cbow.png" width="500" align="left">


(Image from https://lilianweng.github.io/lil-log/2017/10/15/learning-word-embedding.html)

In [50]:
import torch
from torch import nn, optim, tensor, autograd
from torch.nn import functional as F

class CBOW(nn.Module):
    def __init__(self, vocab_size, embd_size, context_size, hidden_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embd_size)
        self.linear1 = nn.Linear(2*context_size*embd_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, inputs):
        embedded = self.embeddings(inputs).view((1, -1))
        hid = F.relu(self.linear1(embedded))
        out = self.linear2(hid)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


## Lets take a closer look from the inputs to the first `nn.Linear`

Cos after it reach the first `nn.Linear` it's just the same as our multi-layered perceptron example =)

In [52]:
# Lets take a look at the first output.
x, y = w2v_dataset[0][0][0],  w2v_dataset[0][0][1], 

x = tensor(x)
y = autograd.Variable(tensor(y, dtype=torch.long))
print(x)
print(y)

tensor([10,  8,  0,  7])
tensor(11)


In [53]:
embd_size = 5
emb = nn.Embedding(len(w2v_dataset.vocab), embd_size)
emb.state_dict()

OrderedDict([('weight', tensor([[ 1.0259,  0.9284, -2.2316,  0.0754, -1.4768],
                      [-1.6520, -1.4818, -0.4483,  0.3457, -0.3436],
                      [-0.2531, -0.1992,  0.7915,  0.1961,  0.2940],
                      ...,
                      [-0.2093,  0.4285, -1.8079,  0.7984,  0.3629],
                      [ 1.6770, -0.8758,  0.6344, -0.0887, -0.1260],
                      [ 0.5117,  1.1197,  0.3272, -0.5028, -1.0432]]))])

In [54]:
print(emb.state_dict()['weight'].shape)
emb.state_dict()['weight']

torch.Size([1388, 5])


tensor([[ 1.0259,  0.9284, -2.2316,  0.0754, -1.4768],
        [-1.6520, -1.4818, -0.4483,  0.3457, -0.3436],
        [-0.2531, -0.1992,  0.7915,  0.1961,  0.2940],
        ...,
        [-0.2093,  0.4285, -1.8079,  0.7984,  0.3629],
        [ 1.6770, -0.8758,  0.6344, -0.0887, -0.1260],
        [ 0.5117,  1.1197,  0.3272, -0.5028, -1.0432]])

In [55]:
print(emb(x).shape)
print(emb(x))

torch.Size([4, 5])
tensor([[-0.7855,  1.9982,  1.8082, -1.2010,  1.1291],
        [ 1.2505,  0.3433, -0.8396, -0.7779,  0.3282],
        [ 1.0259,  0.9284, -2.2316,  0.0754, -1.4768],
        [-0.2633,  1.4716, -0.6771, -0.3750, -0.0344]],
       grad_fn=<EmbeddingBackward>)


In [56]:
print(emb(x).view(1, -1).shape)
emb(x).view(1, -1)

torch.Size([1, 20])


tensor([[-0.7855,  1.9982,  1.8082, -1.2010,  1.1291,  1.2505,  0.3433, -0.8396,
         -0.7779,  0.3282,  1.0259,  0.9284, -2.2316,  0.0754, -1.4768, -0.2633,
          1.4716, -0.6771, -0.3750, -0.0344]], grad_fn=<ViewBackward>)

In [57]:
hidden_size = 100
lin1 = nn.Linear(len(x)*embd_size, hidden_size)
print(lin1.state_dict())

OrderedDict([('weight', tensor([[-0.1593, -0.1181,  0.0052,  ...,  0.0510, -0.0207,  0.1492],
        [ 0.0397, -0.1941,  0.1248,  ..., -0.0058,  0.0773,  0.0470],
        [ 0.1236, -0.0977,  0.1557,  ..., -0.2216, -0.1264, -0.2094],
        ...,
        [-0.0113, -0.1606,  0.1912,  ..., -0.0442,  0.1083, -0.0970],
        [ 0.0369,  0.1836, -0.0503,  ...,  0.2101,  0.0778,  0.0710],
        [-0.0342, -0.0441,  0.2212,  ..., -0.1659, -0.0135, -0.1570]])), ('bias', tensor([-0.2075,  0.1603,  0.0227,  0.1867, -0.1603, -0.0867,  0.0629,  0.0969,
         0.0912, -0.1158,  0.0868, -0.0484,  0.2205,  0.1823,  0.0326,  0.1284,
         0.0667,  0.0550,  0.0826,  0.0422, -0.2216, -0.1200, -0.0927,  0.1577,
         0.0349,  0.1249, -0.0598,  0.1617, -0.0725,  0.1232, -0.0378,  0.0363,
        -0.1645,  0.2122, -0.0617,  0.2212,  0.1321,  0.1082,  0.0190, -0.1899,
        -0.0055,  0.0886, -0.1133,  0.0539,  0.0648,  0.1851, -0.1185,  0.1833,
         0.2210,  0.0527,  0.0450,  0.0052,  0.0745

In [58]:
print(lin1.state_dict()['weight'].shape)
print(lin1.state_dict()['weight'])

torch.Size([100, 20])
tensor([[-0.1593, -0.1181,  0.0052,  ...,  0.0510, -0.0207,  0.1492],
        [ 0.0397, -0.1941,  0.1248,  ..., -0.0058,  0.0773,  0.0470],
        [ 0.1236, -0.0977,  0.1557,  ..., -0.2216, -0.1264, -0.2094],
        ...,
        [-0.0113, -0.1606,  0.1912,  ..., -0.0442,  0.1083, -0.0970],
        [ 0.0369,  0.1836, -0.0503,  ...,  0.2101,  0.0778,  0.0710],
        [-0.0342, -0.0441,  0.2212,  ..., -0.1659, -0.0135, -0.1570]])


In [59]:
print(lin1(emb(x).view(1, -1)).shape)
lin1(emb(x).view(1, -1))

torch.Size([1, 100])


tensor([[-0.3817,  0.9909,  0.7012, -0.3593, -0.0812, -0.6965, -1.2010, -0.4626,
         -0.6172, -0.7549,  0.1895,  0.4077,  1.5212,  0.1513,  0.2127, -0.3394,
          0.7259,  0.7881, -0.1431, -0.6342,  0.4919, -0.1866, -0.0076, -0.6322,
         -0.2157, -0.0477,  1.1727,  0.7505,  0.7963,  0.8121, -1.0275,  1.2920,
          0.2828,  1.0255,  0.9426,  0.5819,  0.2563, -0.0383,  0.0477, -0.3911,
         -0.5229, -0.2008, -0.0466,  0.2601, -0.3392, -0.1089,  0.2009,  0.2212,
          0.7352, -0.2152, -0.3420, -0.4976, -0.4798, -0.8921, -1.1753,  0.5407,
          1.2047, -0.7487,  0.1302,  0.0670,  0.3469, -0.8901,  0.8454, -0.0210,
         -0.0111, -1.3168, -0.1614,  0.4426,  0.7137,  0.3536,  1.2130,  0.1277,
          0.3944,  0.1174,  0.5066, -0.2962,  0.4578, -0.0253, -0.8236,  0.2473,
         -0.0625,  0.3411, -0.1997, -0.3721,  1.3432, -0.0701,  0.5318, -0.4238,
          0.7625, -0.2290, -0.1641, -0.8921,  0.7023, -0.1720,  0.9794, -0.3581,
          0.6254,  0.3997,  

In [60]:
relu = nn.ReLU()
print(relu(lin1(emb(x).view(1, -1))).shape)
relu(lin1(emb(x).view(1, -1)))

torch.Size([1, 100])


tensor([[0.0000, 0.9909, 0.7012, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1895, 0.4077, 1.5212, 0.1513, 0.2127, 0.0000, 0.7259, 0.7881,
         0.0000, 0.0000, 0.4919, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.1727,
         0.7505, 0.7963, 0.8121, 0.0000, 1.2920, 0.2828, 1.0255, 0.9426, 0.5819,
         0.2563, 0.0000, 0.0477, 0.0000, 0.0000, 0.0000, 0.0000, 0.2601, 0.0000,
         0.0000, 0.2009, 0.2212, 0.7352, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.5407, 1.2047, 0.0000, 0.1302, 0.0670, 0.3469, 0.0000, 0.8454,
         0.0000, 0.0000, 0.0000, 0.0000, 0.4426, 0.7137, 0.3536, 1.2130, 0.1277,
         0.3944, 0.1174, 0.5066, 0.0000, 0.4578, 0.0000, 0.0000, 0.2473, 0.0000,
         0.3411, 0.0000, 0.0000, 1.3432, 0.0000, 0.5318, 0.0000, 0.7625, 0.0000,
         0.0000, 0.0000, 0.7023, 0.0000, 0.9794, 0.0000, 0.6254, 0.3997, 0.1924,
         0.1104]], grad_fn=<ThresholdBackward0>)

In [61]:
lin2 = nn.Linear(hidden_size, len(w2v_dataset.vocab))
print(lin2.state_dict()['weight'].shape)
lin2.state_dict()['weight']

torch.Size([1388, 100])


tensor([[-0.0361,  0.0369,  0.0532,  ...,  0.0423,  0.0221, -0.0308],
        [-0.0447,  0.0967,  0.0583,  ...,  0.0839, -0.0752,  0.0878],
        [ 0.0610,  0.0059, -0.0166,  ..., -0.0994, -0.0846,  0.0311],
        ...,
        [ 0.0064,  0.0385,  0.0630,  ..., -0.0633,  0.0055,  0.0551],
        [ 0.0945, -0.0582,  0.0583,  ...,  0.0669, -0.0209,  0.0890],
        [ 0.0218,  0.0901, -0.0164,  ..., -0.0478, -0.0856,  0.0349]])

In [62]:
h_x = relu(lin1(emb(x).view(1, -1)))
print(lin2(h_x).shape)
lin2(h_x)

torch.Size([1, 1388])


tensor([[-0.0706, -0.0801, -0.1645,  ..., -0.2212,  0.1059,  0.0386]],
       grad_fn=<ThAddmmBackward>)

In [63]:
softmax = nn.LogSoftmax(dim=1)
softmax(lin2(h_x)).detach().numpy().tolist()

[[-7.342565059661865,
  -7.352034091949463,
  -7.436441421508789,
  -7.377696990966797,
  -7.01069974899292,
  -7.53016996383667,
  -7.369412899017334,
  -7.332258701324463,
  -7.271763801574707,
  -7.078455924987793,
  -7.277397632598877,
  -7.313171863555908,
  -7.820171356201172,
  -7.720254898071289,
  -6.781040191650391,
  -6.903045177459717,
  -7.287822723388672,
  -7.216547012329102,
  -6.778151988983154,
  -7.384675025939941,
  -7.458211421966553,
  -7.07975959777832,
  -7.00247859954834,
  -7.403648853302002,
  -7.398122787475586,
  -7.428200721740723,
  -7.719222545623779,
  -7.587791919708252,
  -7.422340393066406,
  -6.894408226013184,
  -7.0969061851501465,
  -6.842694282531738,
  -7.381572723388672,
  -7.4432244300842285,
  -7.286935329437256,
  -7.695927143096924,
  -7.322376728057861,
  -7.581524848937988,
  -7.459885120391846,
  -7.195405960083008,
  -7.331758975982666,
  -7.192982196807861,
  -6.9072699546813965,
  -7.411691665649414,
  -7.513472080230713,
  -7.732112

In [64]:
# Select the index with highest softmax probabilities
torch.max(softmax(lin2(h_x)), 1)

(tensor([-6.3551], grad_fn=<MaxBackward0>), tensor([631]))

<a id="section-3-1-4-train-cbow"></a>

# Now, we train the CBOW model for real.

In [65]:
# First we split the data into training and testing.
from sklearn.model_selection import train_test_split

tokenized_text_train, tokenized_text_test = train_test_split(tokenized_text, test_size=0.1, random_state=42)
len(tokenized_text_train), len(tokenized_text_test)

(211, 24)

In [66]:
import torch
from torch import nn, optim, tensor, autograd
from torch.nn import functional as F

class CBOW(nn.Module):
    def __init__(self, vocab_size, embd_size, context_size, hidden_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embd_size)
        self.linear1 = nn.Linear(2*context_size*embd_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, inputs):
        embedded = self.embeddings(inputs).view((1, -1))
        hid = F.relu(self.linear1(embedded))
        out = self.linear2(hid)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


In [67]:
embd_size = 100
learning_rate = 0.003
hidden_size = 100
window_size = 2

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize the dataset.
w2v_dataset = Word2VecText(tokenized_text_train, window_size=window_size, variant='cbow')
vocab_size = len(w2v_dataset.vocab)

criterion = nn.NLLLoss()
model = CBOW(vocab_size, embd_size, window_size, hidden_size).to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

losses = []

model = nn.DataParallel(model)

num_epochs = 100
for _e in tqdm(range(num_epochs)):
    epoch_loss = []
    for sent_idx in range(w2v_dataset._len):
        for w2v_io in w2v_dataset[sent_idx]:
            # Retrieve the inputs and outputs.
            x, y = w2v_io
            x = tensor(x).to(device)
            y = autograd.Variable(tensor(y, dtype=torch.long)).to(device)
            # Zero gradient.
            model.zero_grad()
            # Calculate the log probability of the context embeddings.
            logprobs = model(x)
            # This unsqueeze thing is really a feature/bug... -_-
            loss = criterion(logprobs, y.unsqueeze(0)) 
            loss.backward()
            optimizer.step()
            epoch_loss.append(float(loss))
    # Save model after every epoch.
    torch.save(model.state_dict(), 'cbow_checkpoint_{}.pt'.format(_e))
    losses.append(sum(epoch_loss)/len(epoch_loss))



100%|██████████| 100/100 [06:43<00:00,  4.04s/it]


In [68]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

sns.set_style("darkgrid")
sns.set(rc={'figure.figsize':(12, 8)})

plt.plot(losses)
plt.show()

<Figure size 1200x800 with 1 Axes>

<a id="section-3-1-4-evaluate-cbow"></a>

# Apply and Evaluate the CBOW Model 

In [69]:
from lazyme import color_str

true_positive = 0
all_data = 0
# Iterate through the test sentences. 
for sent in tokenized_text_test:
    # Extract all the CBOW contexts (X) and targets (Y)
    for w2v_io in w2v_dataset._iterator(w2v_dataset.vectorize(sent)):
        # Retrieve the inputs and outputs.
        x, y = w2v_io
        x = tensor(x)
        y = tensor(y)
        
        if -1 in x: # Skip unknown words.
            continue
            
        with torch.no_grad():
            _, prediction =  torch.max(model(x), 1)
        true_positive += int(prediction) == int(y)
        visualize_predictions(x, y, prediction, w2v_dataset.vocab, window_size=window_size)
        all_data += 1

is 		 the problem is essentially this
essentially 	 problem is not this :
this 		 is essentially motivated : if
: 		 essentially this ______ if a
if 		 this : was a word
a 		 : if a word (
word 		 if a more ( or
( 		 a word word or bigram
or 		 word ( as bigram ,
bigram 		 ( or british , or
<unk> 		 , or uncorrelated , or
<unk> 		 , or to etc .
is 		 the web is a vast
a 		 web is in vast re-
vast 		 is a χ2 re- source
re- 		 a vast being source for
source 		 vast re- and for many
the 		 is that the association is
association 	 that the probability is random
is 		 the association between random ,
random 		 association is never , arbitrary
, 		 is random language arbitrary ,
arbitrary 	 random , good , motivated
, 		 , arbitrary ______ motivated or
motivated 	 arbitrary , motivated or pre-
<unk> 		 or pre- p ( r
, 		 ( r ______ a ,
<unk> 		 a , 1977 , p
their 		 however , then methods are
example 	 , for example , from
, 		 for example from from just
from 		 example , and just those
<unk

not 		 corpora are the randomly generated
randomly 	 are not ( generated ,
generated 	 not randomly should , and
, 		 randomly generated ______ and the
and 		 generated , then the hypothesis
the 		 , and the hypothesis test
hypothesis 	 and the hypothesis test con-
<unk> 		 test con- does the fact
<unk> 		 cases are not in section
<unk> 		 of linguistic probability concern the
<unk> 		 the dis- differences between a
<unk> 		 a and for a linguistic
<unk> 		 a linguistic one of a
<unk> 		 reason to reject the relation
between 	 the relation is , for
, 		 relation between ______ for example
for 		 between , for example ,
example 	 , for example , a
, 		 for example of a verb
a 		 example , and verb ’
verb 		 , a sophisticated ’ s
’ 		 a verb have s syntax
s 		 verb ’ s syntax and
syntax 		 ’ s definition and its
<unk> 		 and its are , as
motivated 	 , as represented rather than
rather 		 as motivated rather than arbitrary
than 		 motivated rather not arbitrary .
value 		 the average value

In [70]:
print('Accuracy:', true_positive/all_data)

Accuracy: 0.28085106382978725


<a id="section-3-1-4-load-model"></a>

# Go back to the 10th Epoch

In [71]:
model_10 = CBOW(vocab_size, embd_size, window_size, hidden_size)
model_10 = torch.nn.DataParallel(model_10)
model_10.load_state_dict(torch.load('cbow_checkpoint_10.pt'))
model_10.eval()

DataParallel(
  (module): CBOW(
    (embeddings): Embedding(1303, 100)
    (linear1): Linear(in_features=400, out_features=100, bias=True)
    (linear2): Linear(in_features=100, out_features=1303, bias=True)
  )
)

In [72]:

true_positive = 0
all_data = 0
# Iterate through the test sentences. 
for sent in tokenized_text_test:
    # Extract all the CBOW contexts (X) and targets (Y)
    for w2v_io in w2v_dataset._iterator(w2v_dataset.vectorize(sent)):
        # Retrieve the inputs and outputs.
        x, y = w2v_io
        x = tensor(x)
        y = tensor(y)
        
        if -1 in x: # Skip unknown words.
            continue
            
        with torch.no_grad():
            _, prediction =  torch.max(model_10(x), 1)
        true_positive += int(prediction) == int(y)
        visualize_predictions(x, y, prediction, w2v_dataset.vocab, window_size=window_size)
        all_data += 1

is 		 the problem is essentially this
essentially 	 problem is not this :
this 		 is essentially in : if
: 		 essentially this ______ if a
if 		 this : was a word
a 		 : if a word (
word 		 if a more ( or
( 		 a word word or bigram
or 		 word ( and bigram ,
bigram 		 ( or ) , or
<unk> 		 , or plan , or
<unk> 		 , or to etc .
is 		 the web is a vast
a 		 web is in vast re-
vast 		 is a χ2 re- source
re- 		 a vast of source for
source 		 vast re- and for many
the 		 is that the association is
association 	 that the probability is random
is 		 the association between random ,
random 		 association is random , arbitrary
, 		 is random language arbitrary ,
arbitrary 	 random , and , motivated
, 		 , arbitrary ______ motivated or
motivated 	 arbitrary , to or pre-
<unk> 		 or pre- p ( r
, 		 ( r ______ a ,
<unk> 		 a , random , p
their 		 however , random methods are
example 	 , for example , from
, 		 for example from from just
from 		 example , and just those
<unk> 		 just those a errors t

and 		 generated , then the hypothesis
the 		 , and the hypothesis test
hypothesis 	 and the hypothesis test con-
<unk> 		 test con- of the fact
<unk> 		 cases are random in section
<unk> 		 of linguistic probability concern the
<unk> 		 the dis- linguistics between a
<unk> 		 a and for a linguistic
<unk> 		 a linguistic subset of a
<unk> 		 reason to reject the relation
between 	 the relation random , for
, 		 relation between ) for example
for 		 between , and example ,
example 	 , for example , a
, 		 for example of a verb
a 		 example , and verb ’
verb 		 , a would ’ s
’ 		 a verb have s syntax
s 		 verb ’ s syntax and
syntax 		 ’ s definition and its
<unk> 		 and its english , as
motivated 	 , as a rather than
rather 		 as motivated a than arbitrary
than 		 motivated rather the arbitrary .
value 		 the average value of the
of 		 average value of the error
the 		 value of the error term
error 		 of the same term ,
term 		 the error word , language
, 		 error term are language is
la

In [73]:
print('Accuracy:', true_positive/all_data)

Accuracy: 0.26382978723404255


# [optional] How to Handle Unknown Words? 

This is not the best way to handle unknown words, but we can simply assign an index for unknown words.

**Hint:** Ensure that you have `gensim` version 3.7.0 first. Otherwise this part of the code won't work. Try `python -m pip install -U pip` and then `python -m pip install -U gensim==3.7.0`

In [81]:
vocab = Dictionary(['this is a foo bar sentence'.split()])
dict(vocab.items())

{0: 'a', 1: 'bar', 2: 'foo', 3: 'is', 4: 'sentence', 5: 'this'}

In [82]:
# See https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.patch_with_special_tokens
vocab = Dictionary(['this is a foo bar sentence'.split()])

try:
    special_tokens = {'<pad>': 0, '<unk>': 1}
    vocab.patch_with_special_tokens(special_tokens)
except: # If gensim is not 3.7.0
    pass
    
dict(vocab.items())

{0: 'a', 1: 'bar', 2: 'foo', 3: 'is', 4: 'sentence', 5: 'this'}

# [optional] Lets Rewrite the `Word2VecText` Object

Now with the (i) unknown word patch in the vocabulary as well as (ii) `skipgram_iterator`

In [83]:
class Word2VecText(Dataset):
    def __init__(self, tokenized_texts, window_size, variant):
        """
        :param tokenized_texts: Tokenized text.
        :type tokenized_texts: list(list(str))
        """
        self.sents = tokenized_texts
        self._len = len(self.sents)
        
        # Add the unknown word patch here.
        self.vocab = Dictionary(self.sents)
        try:
            special_tokens = {'<pad>': 0, '<unk>': 1}
            self.vocab.patch_with_special_tokens(special_tokens)
        except:
            pass
        
        self.window_size = window_size
        self.variant = variant
        if variant.lower() == 'cbow':
            self._iterator = self.cbow_iterator
        elif variant.lower() == 'skipgram':
            self._iterator = self.skipgram_iterator

    def __getitem__(self, index):
        """
        The primary entry point for PyTorch datasets.
        This is were you access the specific data row you want.
        
        :param index: Index to the data point.
        :type index: int
        """
        vectorized_sent = self.vectorize(self.sents[index])
        
        return list(self._iterator(vectorized_sent, self.window_size))

    def __len__(self):
        return self._len
    
    def vectorize(self, tokens):
        """
        :param tokens: Tokens that should be vectorized. 
        :type tokens: list(str)
        """
        # See https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2idx 
        return self.vocab.doc2idx(tokens, unknown_word_index=1)
    
    def unvectorize(self, indices):
        """
        :param indices: Converts the indices back to tokens.
        :type tokens: list(int)
        """
        return [self.vocab[i] for i in indices]
    
    def cbow_iterator(self, tokens, window_size):
        n = window_size * 2 + 1
        for window in per_window(tokens, n):
            target = window.pop(window_size)
            yield {'x': window, 'y': target}   # X = window ; Y = target. 
            
    def skipgram_iterator(self, tokens, window_size):
        n = window_size * 2 + 1 
        for i, window in enumerate(per_window(tokens, n)):
            focus = window.pop(window_size)
            # Generate positive samples.
            for context_word in window:
                yield {'x': (focus, context_word), 'y':1}
            # Generate negative samples.
            for _ in range(n-1):
                leftovers = tokens[:i] + tokens[i+n:]
                if leftovers:
                    yield {'x': (focus, random.choice(leftovers)), 'y':0}
                

<a id="section-3-1-5"></a>

# Lets try the skipgram task

In [84]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size, embd_size):
        super(SkipGram, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embd_size)
    
    def forward(self, focus, context):
        embed_focus = self.embeddings(focus).view((1, -1))
        embed_context = self.embeddings(context).view((1, -1))
        # See https://pytorch.org/docs/stable/torch.html#torch.t
        score = torch.mm(embed_focus, torch.t(embed_context))
        log_probs = F.logsigmoid(score)
        return log_probs

<a id="section-3-1-5-foward"></a>

# Take a closer look at what's in the `forward()`

In [85]:
xx1 = torch.rand(1,20)
xx2 = torch.rand(1,20)

xx1_numpy = xx1.detach().numpy()
xx2_numpy = xx2.detach().numpy()

In [86]:
print(xx1_numpy.shape)
print(xx2_numpy.T.shape)
print(np.dot(xx1_numpy, xx2_numpy.T))

(1, 20)
(20, 1)
[[4.799898]]


In [80]:
print(xx1.shape)
print(torch.t(xx2).shape) 

print(torch.mm(xx1, torch.t(xx2))) # 

torch.Size([1, 20])
torch.Size([20, 1])
tensor([[6.4539]])


<a id="section-3-1-5-train"></a>

# Train a Skipgram model (for real)

In [95]:

embd_size = 100
learning_rate = 0.03
hidden_size = 300
window_size = 3

# Initialize the dataset.
w2v_dataset = Word2VecText(tokenized_text_train, window_size=3, variant='skipgram')
vocab_size = len(w2v_dataset.vocab)

criterion = nn.MSELoss()
model = SkipGram(vocab_size, embd_size,).to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

losses = []

model = nn.DataParallel(model)

num_epochs = 100
for _e in tqdm(range(num_epochs)):
    epcoh_loss = 0
    for sent_idx in range(w2v_dataset._len):
        for w2v_io in w2v_dataset[sent_idx]:
            # Retrieve the inputs and outputs.
            x1, x2 = w2v_io['x']
            x1, x2 = tensor(x1).to(device), tensor(x2).to(device)
            y = autograd.Variable(tensor(w2v_io['y'], dtype=torch.float)).to(device)
            # Zero gradient.
            model.zero_grad()
            # Calculate the log probability of the context embeddings.
            logprobs = model(x1, x2)
            # This unsqueeze thing is really a feature/bug... -_-
            loss = criterion(logprobs[0], y.unsqueeze(0)) 
            loss.backward()
            optimizer.step()
            epcoh_loss += float(loss)
    torch.save(model.state_dict(), 'skipgram_checkpoint_{}.pt'.format(_e))
    losses.append(epcoh_loss)



  4%|▍         | 4/100 [02:02<48:51, 30.54s/it]

KeyboardInterrupt: 

<a id="section-3-1-5-evaluate"></a>

# Evaluate the model on the skipgram task

In [96]:

true_positive = 0
all_data = 0
# Iterate through the test sentences. 
for sent in tokenized_text_test:
    # Extract all the CBOW contexts (X) and targets (Y)
    for w2v_io in w2v_dataset._iterator(w2v_dataset.vectorize(sent)):
        model.zero_grad()
        # Retrieve the inputs and outputs.
        x1, x2 = w2v_io['x']
        x1, x2 = tensor(x1), tensor(x2)
        y = w2v_io['y']
        _, prediction =  torch.max(model(x1, x2), 1)    
        true_positive += int(prediction) == int(y)
        all_data += 1

TypeError: skipgram_iterator() missing 1 required positional argument: 'window_size'

In [ ]:
print('Accuracy:', true_positive/all_data)

## Download the Collobert and Weston SENNA Embeddings


If you're on a Mac or Linux, you can use the `!` bang commands in the next cell to get the data.

```
!pip install kaggle
!mkdir -p .kaggle
!echo '{"username":"natgillin","key":"54ae95ab760b52c3307ed4645c6c9b5d"}' > .kaggle/kaggle.json
!chmod 600 .kaggle/kaggle.json
!kaggle datasets download -d alvations/vegetables-senna-embeddings --force -p ./
```

If you're on windows go to https://www.kaggle.com/alvations/vegetables-senna-embeddings and download the data files. 

What's most important are the 
 - `.txt` file that contains the vocabulary list
 - `.npy` file that contains the binarized numpy array
 
The rows of the numpy array corresponds to the vocabulary in the order from the `.txt` file.

In [97]:
!pip install kaggle
!mkdir -p .kaggle
!echo '{"username":"natgillin","key":"54ae95ab760b52c3307ed4645c6c9b5d"}' > .kaggle/kaggle.json
!chmod 600 .kaggle/kaggle.json
!kaggle datasets download -d alvations/vegetables-senna-embeddings --force -p ./

100%|█████████████████████████████████████▉| 64.0M/64.1M [00:07<00:00, 7.11MB/s]
100%|██████████████████████████████████████| 64.1M/64.1M [00:07<00:00, 9.08MB/s]


<a id="section-3-1-6-vocab"></a>


## 3.1.6. Loading Pre-trained Embeddings

Lets overwrite the `Word2VecText` object with the pretrained embeddings. 

Most important thing is the overwrite the `Dictionary` from `gensim` with the vocabulary of the pre-trained embeddings, as such:

```python
        # Loads the pretrained keys. 
        with open('senna.wiki-reuters.lm2.50d.txt') as fin:
            pretrained_keys = {line.strip():i for i, line in enumerate(fin)}
        self.vocab = Dictionary({})
        self.vocab.token2id = pretrained_keys
```


In [98]:
class Word2VecText(Dataset):
    def __init__(self, tokenized_texts, window_size, variant):
        """
        :param tokenized_texts: Tokenized text.
        :type tokenized_texts: list(list(str))
        """
        self.sents = tokenized_texts
        self._len = len(self.sents)
        
        # Loads the pretrained keys. 
        with open('senna.wiki-reuters.lm2.50d.txt') as fin:
            pretrained_keys = {line.strip():i for i, line in enumerate(fin)}
        self.vocab = Dictionary({})
        self.vocab.token2id = pretrained_keys
        
        self.window_size = window_size
        self.variant = variant
        if variant.lower() == 'cbow':
            self._iterator = partial(self.cbow_iterator, window_size=self.window_size)
        elif variant.lower() == 'skipgram':
            self._iterator = partial(self.skipgram_iterator, window_size=self.window_size)

    def __getitem__(self, index):
        """
        The primary entry point for PyTorch datasets.
        This is were you access the specific data row you want.
        
        :param index: Index to the data point.
        :type index: int
        """
        vectorized_sent = self.vectorize(self.sents[index])
        
        return list(self._iterator(vectorized_sent))

    def __len__(self):
        return self._len
    
    def vectorize(self, tokens):
        """
        :param tokens: Tokens that should be vectorized. 
        :type tokens: list(str)
        """
        # See https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2idx 
        return self.vocab.doc2idx(tokens, unknown_word_index=-1)
    
    def unvectorize(self, indices):
        """
        :param indices: Converts the indices back to tokens.
        :type tokens: list(int)
        """
        return [self.vocab[i] for i in indices]
    
    def cbow_iterator(self, tokens, window_size):
        n = window_size * 2 + 1
        for window in per_window(tokens, n):
            target = window.pop(window_size)
            yield {'x': window, 'y': target}   # X = window ; Y = target. 
            
    def skipgram_iterator(self, tokens, window_size):
        n = window_size * 2 + 1 
        for i, window in enumerate(per_window(tokens, n)):
            focus = window.pop(window_size)
            # Generate positive samples.
            for context_word in window:
                yield {'x': (focus, context_word), 'y':1}
            # Generate negative samples.
            for _ in range(n-1):
                leftovers = tokens[:i] + tokens[i+n:]
                if leftovers:
                    yield {'x': (focus, random.choice(leftovers)), 'y':0}
                

<a id="section-3-1-6-pretrained"></a>

## Override the embeddings layer with the pre-trained weights.

In PyTorch, the weights of the `nn.Embedding` object can be easily overwritten with `from_pretrained` function, see https://pytorch.org/docs/stable/nn.html#embedding

In [99]:
class SkipGram(nn.Module):
    def __init__(self, pretrained_npy):
        super(SkipGram, self).__init__()
        self.embeddings = nn.Embedding.from_pretrained(pretrained_npy)
    
    def forward(self, focus, context):
        embed_focus = self.embeddings(focus).view((1, -1))
        embed_context = self.embeddings(context).view((1, -1))
        # See https://pytorch.org/docs/stable/torch.html#torch.t
        score = torch.mm(embed_focus, torch.t(embed_context))
        log_probs = F.logsigmoid(score)
        return log_probs

In [101]:
w2v_dataset = Word2VecText(tokenized_text_train, window_size=window_size, variant='skipgram')
pretrained_npy = torch.tensor(np.load('senna.wiki-reuters.lm2.50d.npy'))
pretrained_model = SkipGram(pretrained_npy)

<a id="section-3-1-6-eval-skipgram"></a>
## Test Pretrained Embeddings on the Skipgram Task

In [102]:
true_positive = 0
all_data = 0
# Iterate through the test sentences. 
for sent in tokenized_text_test:
    # Extract all the CBOW contexts (X) and targets (Y)
    for w2v_io in w2v_dataset._iterator(w2v_dataset.vectorize(sent)):
        pretrained_model.zero_grad()
        # Retrieve the inputs and outputs.
        x1, x2 = w2v_io['x']
        if -1 in (x1, x2): # Skip unknown words.
            continue
        x1, x2 = tensor(x1), tensor(x2)
        y = w2v_io['y']
        with torch.no_grad():
            logprobs = pretrained_model(x1, x2)
            _, prediction =  torch.max(logprobs, 1)    
        true_positive += int(prediction) == int(y)
        all_data += 1

In [103]:
with open('senna.wiki-reuters.lm2.50d.txt') as fin:
    pretrained_keys = {line.strip():i for i, line in enumerate(fin)}

In [104]:
print('Accuracy:', true_positive/all_data)

Accuracy: 0.4959667562943046


<a id="section-3-1-6-eval-cbow"></a>
## Test Pretrained Embeddings on the CBOW Task

In [105]:
class CBOW(nn.Module):
    def __init__(self, pretrained_npy, context_size, hidden_size):
        super(CBOW, self).__init__()
        vocab_size, embd_size = list(pretrained_npy.shape)
        self.embeddings = nn.Embedding.from_pretrained(pretrained_npy)
        self.linear1 = nn.Linear(2*context_size*embd_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, inputs):
        embedded = self.embeddings(inputs).float().view((1, -1))
        hid = F.relu(self.linear1(embedded))
        out = self.linear2(hid)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


In [106]:
window_size = 5
w2v_dataset = Word2VecText(tokenized_text_train, window_size=window_size, variant='cbow')
hidden_size = 300
pretrained_cbow_model = CBOW(pretrained_npy, window_size, hidden_size)

In [107]:

true_positive = 0
all_data = 0
# Iterate through the test sentences. 
for sent in tokenized_text_test:
    # Extract all the CBOW contexts (X) and targets (Y)
    for w2v_io in w2v_dataset._iterator(w2v_dataset.vectorize(sent)):
        # Retrieve the inputs and outputs.
        x = tensor(w2v_io['x'])
        y = tensor(w2v_io['y'])
        
        if -1 in x: # Skip unknown words.
            continue
        with torch.no_grad():
            _, prediction =  torch.max(pretrained_cbow_model(x), 1)
        true_positive += int(prediction) == int(y)
        visualize_predictions(x, y, prediction, w2v_dataset.vocab, window_size=window_size)
        all_data += 1

: 		 the problem is essentially this mahler if a word ( or
re- 		 the web is a vast luckless source for many languages .
random 		 is that the association is knocked , arbitrary , motivated or
, 		 that the association is random mansion arbitrary , motivated or pre-
<unk> 		 arbitrary , motivated or pre- pegmatite ( r , a ,
inevitably 	 however , their methods are new-home noisy , suffering , for
noisy 		 , their methods are inevitably tobago , suffering , for example
, 		 their methods are inevitably noisy justice suffering , for example ,
suffering 	 methods are inevitably noisy , sidereal , for example , from
, 		 are inevitably noisy , suffering homesteading for example , from just
for 		 inevitably noisy , suffering , j.c. example , from just those
example 	 noisy , suffering , for relegation , from just those parser
, 		 , suffering , for example callers from just those parser errors
from 		 suffering , for example , condottieri just those parser errors that
just 		 , for example

distinct 	 , but that is a hausmann issue : wherever there is
issue 		 but that is a distinct schools.he : wherever there is enough
: 		 that is a distinct issue racially wherever there is enough data
wherever 	 is a distinct issue : stews there is enough data ,
there 		 a distinct issue : wherever hoogovens is enough data , it
is 		 distinct issue : wherever there tracers enough data , it is
enough 		 issue : wherever there is ingersoll data , it is rejected
data 		 : wherever there is enough newts , it is rejected .
are 		 since words in a text ayacucho not random , we know
not 		 words in a text are agrippa random , we know that
random 		 in a text are not sidi , we know that our
, 		 a text are not random zarb we know that our corpora
we 		 text are not random , indigenisation know that our corpora are
know 		 are not random , we hiddink that our corpora are not
that 		 not random , we know farragut our corpora are not randomly
our 		 random , we know that surendra corpora are not 

In [108]:
print('Accuracy:', true_positive/all_data)

Accuracy: 0.0


<a id="section-3-1-6-unfreeze-finetune"></a>
## Unfreeze the Embedddings and Tune it on the CBOW Task

In [109]:
class CBOW(nn.Module):
    def __init__(self, pretrained_npy, context_size, hidden_size):
        super(CBOW, self).__init__()
        vocab_size, embd_size = list(pretrained_npy.shape)
        self.embeddings = nn.Embedding.from_pretrained(pretrained_npy, freeze=False)
        self.linear1 = nn.Linear(2*context_size*embd_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, inputs):
        embedded = self.embeddings(inputs).float().view((1, -1))
        hid = F.relu(self.linear1(embedded))
        out = self.linear2(hid)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


In [110]:
window_size = 2 
w2v_dataset = Word2VecText(tokenized_text_train, window_size=window_size, variant='cbow')
hidden_size = 300
pretrained_cbow_model = CBOW(pretrained_npy, window_size, hidden_size)

In [111]:
learning_rate = 0.003
criterion = nn.NLLLoss()
optimizer = optim.SGD(pretrained_cbow_model.parameters(), lr=learning_rate)

losses = []

model = nn.DataParallel(pretrained_cbow_model)

num_epochs = 100
for _e in tqdm(range(num_epochs)):
    epoch_loss = []
    for sent_idx in range(w2v_dataset._len):
        for w2v_io in w2v_dataset[sent_idx]:
            # Retrieve the inputs and outputs.
            x = tensor(w2v_io['x'])
            y = autograd.Variable(tensor(w2v_io['y'], dtype=torch.long))
            
            if -1 in x or int(y) == -1:
                continue
            # Zero gradient.
            model.zero_grad()
            # Calculate the log probability of the context embeddings.
            logprobs = pretrained_cbow_model(x)
            # This unsqueeze thing is really a feature/bug... -_-
            loss = criterion(logprobs, y.unsqueeze(0)) 
            loss.backward()
            optimizer.step()
            epoch_loss.append(float(loss))
    # Save model after every epoch.
    torch.save(model.state_dict(), 'cbow_finetuning_checkpoint_{}.pt'.format(_e))
    losses.append(sum(epoch_loss)/len(epoch_loss))




  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

<a id="section-3-1-6-reval-cbow"></a>

## Re-Test Pretrained Embeddings on the CBOW Task

In [ ]:

true_positive = 0
all_data = 0
# Iterate through the test sentences. 
for sent in tokenized_text_test:
    # Extract all the CBOW contexts (X) and targets (Y)
    for w2v_io in w2v_dataset._iterator(w2v_dataset.vectorize(sent)):
        # Retrieve the inputs and outputs.
        x = tensor(w2v_io['x'])
        y = tensor(w2v_io['y'])
        
        if -1 in x: # Skip unknown words.
            continue
        with torch.no_grad():
            _, prediction =  torch.max(pretrained_cbow_model(x), 1)
        true_positive += int(prediction) == int(y)
        visualize_predictions(x, y, prediction, w2v_dataset.vocab, window_size=window_size)
        all_data += 1

In [ ]:
print('Accuracy:', true_positive/all_data)